In [4]:
import datetime
import fetcher
import compute
import pandas as pd
from finta import TA
import numpy as np

startDate = datetime.date(2020, 1, 1)
#endDate = datetime.date(2024, 8, 20)
endDate = datetime.date.today()
Stocks_list = []#"NVDA", "AAPL", "DKNG", "UBER"

# Retrieving tickers from a file
stocks = pd.read_csv("SCANS\\US_2024-08-23.csv")
#Neglecting tickers with Exchange values "AMEX" and "OTC"
stocks = stocks[~stocks["Exchange"].isin(["OTC", "AMEX"])]
#stocks = stocks[~stocks["Industry"].isin(["Biotechnology", "Pharmaceuticals: major", "Medical Specialties"])]
for ticker in stocks["Symbol"].tolist()[1:]:
    Stocks_list.append(ticker)

df_index = fetcher.__download_data( ["SPY"], startDate, endDate, "US")

info = {}
RS_scores = {}

focus_list_IPO = pd.DataFrame(columns=['ticker','ADR','ADV'])
for ticker in Stocks_list:
    
    #Neglecting tickers with .A or .B or "/" in their names
    if len(ticker.split(".")) >1 or len(ticker.split("/")) >1:
        continue
    df = fetcher.__download_data([ticker], startDate, endDate, "US")
    
    ## Having this in the beginning is to have all the stocks while calculating percentile
    RS_score = compute.RS_score(df["Close"], df_index["Close"])
    RS_scores[ticker] = RS_score 
    
    # IPO stocks in the last 3 years
    if len(df) < 252*1:
        # Avg Volume*Price
        if len(df) > 30:
            ADV = compute.avg_dollar_vol(df, 30)
        
                
        # ADR%
        if len(df) > 20:
            ADR = compute.ADR(df, 20)
            
            new_row = {"ticker":ticker, "ADR": ADR, "ADV": ADV}
            focus_list_IPO = focus_list_IPO.append(new_row, ignore_index=True)
    else:
        # Avg Volume*Price
        ADV = compute.avg_dollar_vol(df, 30)

        # ADR%
        ADR = compute.ADR(df, 20)
        
        #52 week highs
        _52weekHigh = max(df["Close"][-252:])

        # Small pullback calculation -> Close must be 5% or more below 21 day High
        _20DayHigh = max(df["Close"][-21:])

        # Relative Volume
        RVol = df.iloc[-1]["Volume"]/ADV


        # RS
        #RS = compute.RS_line(df, df_index, 252)
        EMA10 = TA.EMA(df, period= 10)
        EMA21 = TA.EMA(df, period= 21)
        SMA50 =  np.convolve(df["Close"], np.ones(50)/50, mode='valid')
        SMA200 = np.convolve(df["Close"], np.ones(200)/200, mode='valid')
        slope50 = compute.linear_regression(SMA50[-6:])
        slope200 = compute.linear_regression(SMA200[-6:])
        #compute.calc_slope(SMA[-42:], 42)
        #print(angle)
        info[ticker] = {"Close": df["Close"][-1], "ADR" : ADR, "ADV" : ADV, "RVol": RVol, "52weekHigh":_52weekHigh, "20DayHigh": _20DayHigh, "EMA10": EMA10[-1], "EMA21": EMA21[-1], "SMA50": SMA50[-1], "SMA200": SMA200[-1], "Slope50": slope50, "Slope200": slope200}
        #RS_rating_test[ticker] = compute.RS_rating_new(RS_scores)


# Sorting based on RS_score 
RS_scores = {ticker: RS_score for ticker, RS_score in sorted(RS_scores.items(), key=lambda item: item[1])}


# RS_rating for the stocks
RS_ratings = compute.RS_rating(RS_scores)

# Storing all info in a dataframe
focus = pd.DataFrame(columns=['ticker',"Close", 'ADR','ADV', "RVol", '52weekHigh', "20DayHigh",'RS_ratings', "EMA10", "EMA21","SMA50", "SMA200", "Slope50", "Slope200"])
for ticker, values in info.items():
    new_row = {"ticker":ticker, "Close" : values["Close"], "ADR": values["ADR"], "ADV": values["ADV"], "RVol": values["RVol"], '52weekHigh' : values["52weekHigh"], "20DayHigh" : values["20DayHigh"], "RS_ratings": RS_ratings[ticker], "EMA10":values["EMA10"], "EMA21":values["EMA21"], "SMA50": values["SMA50"], "SMA200": values["SMA200"], "Slope50": values["Slope50"], "Slope200": values["Slope200"]}
    focus = focus.append(new_row, ignore_index=True)
    



fetching data from API for MSFT
[*********************100%***********************]  1 of 1 completed
fetching data from API for CXM
[*********************100%***********************]  1 of 1 completed
fetching data from API for FRSH
[*********************100%***********************]  1 of 1 completed
fetching data from API for CCCS
[*********************100%***********************]  1 of 1 completed
fetching data from API for ZETA
[*********************100%***********************]  1 of 1 completed
fetching data from API for GEN
[*********************100%***********************]  1 of 1 completed
fetching data from API for LYFT
[*********************100%***********************]  1 of 1 completed
fetching data from API for BSY
[*********************100%***********************]  1 of 1 completed
fetching data from API for LIF
[*********************100%***********************]  1 of 1 completed
fetching data from API for EH
[*********************100%***********************]  1 of 1 comple

### Filtering IPO list

In [5]:
final_focus_IPO = focus_list_IPO.loc[(focus['ADR'] >=2.75) & (focus['ADV'] >=25000000)]
len(final_focus_IPO)

5

### Filtering NON IPO stocks

In [13]:
#RS_rating of 70 here is equivalent to ~85 in tradingview
final_focus_NON_IPO = focus.loc[ (focus['ADR'] >=2.75) & (focus['ADV'] >=25000000) & (focus['ADV'] <1000000000) & (focus['RVol'] <=1) & (focus["Close"] <= .99*focus['20DayHigh']) & (focus["RS_ratings"] >= 70) &(focus["Close"] >= focus['SMA200'])  & (focus['SMA50'] >=focus['SMA200']) & (focus['Close'] >=.9*focus['52weekHigh'])] #&  &  & (focus['Slope200'] >=0)(focus['RS_ratings'] >= 40) & (focus['Slope50'] >=0) & (focus['Slope200'] >=0)
len(final_focus_NON_IPO["ticker"].values)

115

In [14]:
final_focus_NON_IPO_large_ADV = focus.loc[ (focus['ADR'] >=2.25) & (focus['ADV'] >=1000000000) &(focus["Close"] >= focus['SMA200'])  & (focus['SMA50'] >=focus['SMA200']) & (focus['Close'] >=.8*focus['52weekHigh'])]#& (focus["EMA10"] >= focus['SMA50']) & (focus["EMA21"] >= focus['SMA50'])
len(final_focus_NON_IPO_large_ADV)

18

In [15]:
final_focus_NON_IPO = pd.concat([final_focus_NON_IPO, final_focus_NON_IPO_large_ADV], axis=0)

### Sorting

In [16]:
final_focus_NON_IPO = pd.DataFrame(final_focus_NON_IPO.sort_values(by=['ADR'], ascending=False).to_numpy(), 
                   index=final_focus_NON_IPO.index, columns=final_focus_NON_IPO.columns)

In [17]:
"""final_focus_NON_IPO = pd.DataFrame(final_focus_NON_IPO.sort_values(by=['RS_ratings'], ascending=True).to_numpy(), 
                   index=final_focus_NON_IPO.index, columns=final_focus_NON_IPO.columns)"""

"final_focus_NON_IPO = pd.DataFrame(final_focus_NON_IPO.sort_values(by=['RS_ratings'], ascending=True).to_numpy(), \n                   index=final_focus_NON_IPO.index, columns=final_focus_NON_IPO.columns)"

In [18]:
stocks = stocks.replace({'/': '_'}, regex=True)

### Grouping by Sectors & Industries

In [19]:
sectors = {}
industries = {}
for index, row in final_focus_NON_IPO.iterrows():
    exchange = stocks.loc[stocks['Symbol'] == row["ticker"]]["Exchange"].values[0]
    sector = stocks.loc[stocks['Symbol'] == row["ticker"]]["Sector"].values[0]
    #print(sector)
    industry = stocks.loc[stocks['Symbol'] == row["ticker"]]["Industry"].values[0]
    #print(industry)
    if sector not in sectors.keys():
        sectors[sector] = {}
        
    if industry not in sectors[sector].keys():
        #print(industry)
        sectors[sector][industry] = []
    
    #print(sectors)
            #print(sectors[sector])
    sectors[sector][industry].append(row["ticker"])


In [20]:
sectors

{'Health technology': {'Medical specialties': ['CDNA',
   'VCYT',
   'PRCT',
   'NTRA',
   'GMED'],
  'Pharmaceuticals: major': ['TGTX',
   'MIRM',
   'RARE',
   'ALNY',
   'LLY',
   'UTHR',
   'EXEL'],
  'Biotechnology': ['ADMA']},
 'Process industries': {'Industrial specialties': ['ASPN'],
  'Agricultural commodities_Milling': ['PPC']},
 'Electronic technology': {'Aerospace & defense': ['RKLB', 'CR'],
  'Semiconductors': ['NVDA', 'MPWR', 'AVGO', 'FN', 'TSM', 'CRUS'],
  'Electronic equipment_Instruments': ['COHR', 'NVMI', 'ITRI'],
  'Computer peripherals': ['ANET', 'STX'],
  'Telecommunications equipment': ['AAPL']},
 'Retail trade': {'Specialty stores': ['CVNA', 'GPI'],
  'Food retail': ['SFM'],
  'Department stores': ['OLLI', 'PDD'],
  'Internet retail': ['MELI', 'AMZN', 'BABA'],
  'Apparel_Footwear retail': ['BURL'],
  'Home improvement chains': ['HD']},
 'Consumer services': {'Restaurants': ['SG', 'EAT', 'SHAK'],
  'Other consumer services': ['LTH', 'ATGE', 'LRN', 'HRB', 'BKNG'],


In [21]:
bam = []
industries_not_interested = []# "Finance", "Health technology"
for sector,values in sectors.items():
    if sector not in industries_not_interested:
        for industry in values.values():
            for ticker in list(industry):
                bam.append(ticker)


In [22]:
final_focus_NON_IPO.loc[final_focus_NON_IPO["ticker"] == "LVMH"]

,ticker,Close,ADR,ADV,RVol,52weekHigh,20DayHigh,RS_ratings,EMA10,EMA21,SMA50,SMA200,Slope50,Slope200


In [23]:
print(bam)

['CDNA', 'VCYT', 'PRCT', 'NTRA', 'GMED', 'TGTX', 'MIRM', 'RARE', 'ALNY', 'LLY', 'UTHR', 'EXEL', 'ADMA', 'ASPN', 'PPC', 'RKLB', 'CR', 'NVDA', 'MPWR', 'AVGO', 'FN', 'TSM', 'CRUS', 'COHR', 'NVMI', 'ITRI', 'ANET', 'STX', 'AAPL', 'CVNA', 'GPI', 'SFM', 'OLLI', 'PDD', 'MELI', 'AMZN', 'BABA', 'BURL', 'HD', 'SG', 'EAT', 'SHAK', 'LTH', 'ATGE', 'LRN', 'HRB', 'BKNG', 'SPHR', 'RCL', 'CDE', 'HL', 'IAG', 'KGC', 'AGI', 'EGO', 'GOLD', 'WPM', 'AEM', 'CRS', 'ATI', 'LPX', 'MOD', 'LII', 'MLI', 'ATMU', 'HA', 'UBER', 'MATX', 'APP', 'ZETA', 'PLTR', 'YOU', 'MNDY', 'TTD', 'CVLT', 'CWAN', 'PANW', 'FICO', 'NOW', 'CRDO', 'NU', 'WIX', 'VRNS', 'FTNT', 'MMYT', 'SPOT', 'META', 'NFLX', 'GOOGL', 'GOOG', 'FIX', 'MTZ', 'PRIM', 'DY', 'VIST', 'VNOM', 'FOUR', 'BCO', 'MHO', 'CCS', 'SKY', 'TOL', 'FTDR', 'MTH', 'TPH', 'PHM', 'SN', 'TBBK', 'BFH', 'WAL', 'GBCI', 'PNFP', 'ALLY', 'STEP', 'TPG', 'KKR', 'JXN', 'URI', 'SLG', 'PIPR', 'LAZ', 'MC', 'VIRT', 'PJT', 'GS', 'JLL', 'KEY', 'BKU', 'ZION', 'UMBF', 'NRG', 'FYBR', 'TPL']


In [70]:
industries_to_concentrate = ['Telecommunications equipment',
 'Aerospace & defense',
 'Homebuilding',
 'Data processing services',
 'Specialty stores',
 'Home improvement chains',
 'Food retail',
 'Pharmaceuticals: other',
 'Pharmaceuticals: generic',
 'Financial publishing/Services',
 'Managed health care',
 'Hospital/Nursing management',
 'Services to the health industry',
 'Medical/Nursing services',
 'Major banks',
 'Investment banks/Brokers',
 'Investment managers',
 'Finance/Rental/Leasing',
 'Multi-line insurance',
 'Real estate investment trusts',
 'Regional banks',
 'Property/Casualty insurance',
 'Insurance brokers/Services',
 'Life/Health insurance',
 'Real estate development',
 'Specialty insurance',
 'Building products',
 'Metal fabrication',
 'Office equipment/Supplies',
 'Air freight/Couriers',
 'Tobacco',
 'Broadcasting',
 'Publishing: newspapers',
 'Media conglomerates',
 'Publishing: books/magazines',
 'Wireless telecommunications',
 'Specialty telecommunications',
 'Gas distributors',
 'Water utilities',
 'Alternative power generation',
 'Chemicals: specialty',
 'Industrial specialties',
 'Pulp & paper',
 'Precious metals',
 'Medical distributors',
 'Environmental services',
 'Oil & gas pipelines',
 'Investment trusts/Mutual funds',
 'Miscellaneous']

In [71]:
final_final_focus_industries = {}
for sector in sectors.values():
    for industry, tickers in sector.items():
        if industry in industries_to_concentrate:
            if industry not in final_final_focus_industries.keys():
                final_final_focus_industries[industry] = []
            for ticker in tickers:
                final_final_focus_industries[industry].append(ticker)


In [72]:
final_final_focus_industries

{'Telecommunications equipment': ['AAPL'],
 'Aerospace & defense': ['HWM', 'AXON', 'FTAI', 'ERJ'],
 'Investment managers': ['KKR'],
 'Major banks': ['CFG', 'KEY'],
 'Real estate investment trusts': ['IRM', 'EXR', 'SLG', 'VNO'],
 'Regional banks': ['FCNCA', 'MTB', 'ALLY', 'WAL', 'SNV', 'EWBC'],
 'Real estate development': ['CBRE', 'JLL'],
 'Pharmaceuticals: other': ['ARGX', 'RVMD'],
 'Pharmaceuticals: generic': ['TEVA'],
 'Specialty stores': ['CVNA'],
 'Food retail': ['SFM'],
 'Homebuilding': ['DHI', 'LEN', 'PHM', 'TOL', 'KBH', 'TMHC'],
 'Precious metals': ['GOLD', 'NEM', 'AEM', 'KGC'],
 'Specialty telecommunications': ['ASTS'],
 'Building products': ['MHK'],
 'Environmental services': ['CLH']}

In [73]:
final_final_focus = []
for industry in list(final_final_focus_industries.values()):
    for ticker in industry:
        final_final_focus.append(ticker)



In [74]:
final_final_focus

['AAPL',
 'HWM',
 'AXON',
 'FTAI',
 'ERJ',
 'KKR',
 'CFG',
 'KEY',
 'IRM',
 'EXR',
 'SLG',
 'VNO',
 'FCNCA',
 'MTB',
 'ALLY',
 'WAL',
 'SNV',
 'EWBC',
 'CBRE',
 'JLL',
 'ARGX',
 'RVMD',
 'TEVA',
 'CVNA',
 'SFM',
 'DHI',
 'LEN',
 'PHM',
 'TOL',
 'KBH',
 'TMHC',
 'GOLD',
 'NEM',
 'AEM',
 'KGC',
 'ASTS',
 'MHK',
 'CLH']

In [1]:
sectors_IPO = {}
for index, row in final_focus_IPO.iterrows():
    exchange = stocks.loc[stocks['Symbol'] == row["ticker"]]["Exchange"].values[0]
    sector = stocks.loc[stocks['Symbol'] == row["ticker"]]["Sector"].values[0]
    #print(sector)
    industry = stocks.loc[stocks['Symbol'] == row["ticker"]]["Industry"].values[0]
    #print(industry)
    if sector not in sectors_IPO.keys():
        sectors_IPO[sector] = {}
        
    if industry not in sectors_IPO[sector].keys():
        #print(industry)
        sectors_IPO[sector][industry] = []
    
    #print(sectors_IPO)
            #print(sectors_IPO[sector])
    sectors_IPO[sector][industry].append(row["ticker"])


NameError: name 'final_focus_IPO' is not defined

In [67]:
sectors_IPO

{'Transportation': {'Air freight_Couriers': ['CART'], 'Trucking': ['PAL']},
 'Industrial services': {'Contract drilling': ['SDRL']},
 'Technology services': {'Data processing services': ['CORZ'],
  'Packaged software': ['LIF', 'ULS']},
 'Electronic technology': {'Electronic equipment_Instruments': ['NXT', 'DBD'],
  'Semiconductors': ['ALAB', 'ARM']},
 'Finance': {'Financial conglomerates': ['MSDL'], 'Regional banks': ['NBBK']},
 'Consumer durables': {'Homebuilding': ['SDHC']},
 'Consumer services': {'Hotels_Resorts_Cruise lines': ['ATAT'],
  'Movies_Entertainment': ['MSGE'],
  'Restaurants': ['CAVA']},
 'Energy minerals': {'Integrated oil': ['TXO']},
 'Health technology': {'Medical specialties': ['GRAL'],
  'Pharmaceuticals: major': ['SLRN', 'BHVN']},
 'Health services': {'Medical_Nursing services': ['SOLV']},
 'Non-energy minerals': {'Precious metals': ['TFPM']},
 'Distribution services': {'Wholesale distributors': ['GCT']}}

In [68]:
bam_IPO = []
industries_not_interested = []
for sector,values in sectors_IPO.items():
    if sector not in industries_not_interested:
        for industry in values.values():
            for ticker in list(industry):
                bam_IPO.append(ticker)

In [69]:
bam_IPO

['CART',
 'PAL',
 'SDRL',
 'CORZ',
 'LIF',
 'ULS',
 'NXT',
 'DBD',
 'ALAB',
 'ARM',
 'MSDL',
 'NBBK',
 'SDHC',
 'ATAT',
 'MSGE',
 'CAVA',
 'TXO',
 'GRAL',
 'SLRN',
 'BHVN',
 'SOLV',
 'TFPM',
 'GCT']

### Industries in each sector

In [5]:
a = stocks['Industry'].unique()
#for index, row in stocks.iterrows():
sectors = {}
for industry in a:
    sector = stocks.loc[stocks['Industry'] == industry]["Sector"].values[0]
    if sector not in sectors:
        sectors[sector] = []
    sectors[sector].append(industry)

In [13]:
sectors

{'Utilities': ['Gas distributors',
  'Alternative power generation',
  'Water utilities',
  'Electric utilities'],
 'Industrial services': ['Oil & gas pipelines',
  'Oilfield services/Equipment',
  'Environmental services',
  'Engineering & construction',
  'Contract drilling'],
 'Health technology': ['Biotechnology',
  'Pharmaceuticals: other',
  'Pharmaceuticals: major',
  'Medical specialties',
  'Pharmaceuticals: generic'],
 'Finance': ['Real estate development',
  'Investment banks/Brokers',
  'Investment managers',
  'Real estate investment trusts',
  'Insurance brokers/Services',
  'Regional banks',
  'Property/Casualty insurance',
  'Finance/Rental/Leasing',
  'Financial conglomerates',
  'Major banks',
  'Multi-line insurance',
  'Savings banks',
  'Life/Health insurance',
  'Specialty insurance'],
 'Transportation': ['Marine shipping',
  'Trucking',
  'Air freight/Couriers',
  'Other transportation',
  'Airlines',
  'Railroads'],
 'Technology services': ['Packaged software',


In [ ]:
a = {"Materials":["Process industries", "Non-energy minerals"], "Energy":["Energy minerals", 'Oil & gas pipelines', 'Oilfield services/Equipment'], "Industrials": ['Producer manufacturing', 'Industrial services', 'Aerospace & defense', 'Electronic technology', 'Transportation'], "Consumer Discretionary":['Consumer durables', 'Consumer services', 'Retail trade', 'Distribution services'], "Consumer Staples":['Consumer non-durables'], "Health Care":['Health technology', 'Health services']
     , "Financials": ['Finance'], "Information Technology":['Technology services','Semiconductors'], "Communication Services":['Communications'], "Utilities":['Utilities'],"Real Estate":[] }

In [52]:
sectors["Non-energy minerals"]["Precious metals"]

['GOLD',
 'NEM',
 'AEM',
 'KGC',
 'WPM',
 'FNV',
 'GFI',
 'RGLD',
 'HMY',
 'AU',
 'IAG',
 'AGI',
 'EGO']